In [1]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
)
from transformers.pipelines import AggregationStrategy
from transformers import pipeline
import torch
import numpy as np
import json
# import pandas as pd
import pprint
pp = pprint.PrettyPrinter()

/home/eboraks/miniconda3/envs/icog/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'sentence_transformers'

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
st_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

In [14]:
vector = st_model.encode("places")
vector.shape

(384,)

In [2]:
# pegasus_summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail")

pszemraj_summarizer = pipeline("summarization", model="pszemraj/led-base-book-summary") 

In [5]:
path_file = '/home/eboraks/Projects/icognition_backend/data/icog_pages/bergum.medium.com%2Ffour-mistakes-when-introducing-embeddings-and-vector-search-d39478a568c5.json'
with open(path_file, 'r') as f:
    page = json.load(f)


In [22]:
def get_paragraphs(page) -> list:
    for k, v in page['paragraphs'].items():
        print(v)

In [6]:
paragaphs = [v for k, v in page['paragraphs'].items()]
print(page['url'])


https://bergum.medium.com/four-mistakes-when-introducing-embeddings-and-vector-search-d39478a568c5#


In [20]:
def summarize_text(text, token_max_length=512, summary_min_length=80, summary_max_length=120) -> str:
    """This method summarizes text.

    Args:
        text (str): text from which the summary should be generated
    """
    model = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')
    tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')
    print(tokenizer.model_max_length)

    tokens_input = tokenizer.encode(
        "summarize: " + text, return_tensors='pt', max_length=token_max_length, truncation=True)
    ids = model.generate(
        tokens_input, min_length=summary_min_length, max_length=summary_max_length)
    summary = tokenizer.decode(ids[0], skip_special_tokens=True)

    return summary

In [21]:
model = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-xsum')
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-xsum')
print(tokenizer.model_max_length)


512


In [7]:
class TransformersTextSummarizer():
    def __init__ (self, model_key = 'facebook/bart-large-cnn', language = 'en'):
        self._tokenizer = AutoTokenizer.from_pretrained(model_key)

        self._language = language

        self._model = AutoModelForSeq2SeqLM.from_pretrained(model_key)

        self._device = "cuda:0" if torch.cuda.is_available() else "cpu"

    def __chunk_text(self, paragraphs):
        
        chunks = []

        chunk = ''

        length = 0

        for paragraph in paragraphs:
            tokenized_sentence = self._tokenizer.encode(paragraph, truncation=False, max_length=None, return_tensors='pt') [0]

            if len(tokenized_sentence) > self._tokenizer.model_max_length:
                continue

            length += len(tokenized_sentence)

            if length <= self._tokenizer.model_max_length:
                chunk = chunk + ' ' + paragraph
            else:
                chunks.append(chunk.strip())
                chunk = paragraph
                length = len(tokenized_sentence)

        if len(chunk) > 0:
            chunks.append(chunk.strip())

        return chunks

    def __clean_text(self, text):
      if text.count('.') == 0:
        return text.strip()

      end_index = text.rindex('.') + 1

      return text[0 : end_index].strip()

    def summarize(self, text, beams=5, *args, **kwargs):
        chunk_texts = self.__chunk_text(text)

        chunk_summaries = []

        for chunk_text in chunk_texts:
            input_tokenized = self._tokenizer.encode(chunk_text, return_tensors='pt')

            input_tokenized = input_tokenized.to(self._device)

            summary_ids = self._model.to(self._device).generate(
                input_tokenized, 
                length_penalty=3.0, 
                min_length = int(0.1 * len(chunk_text)), 
                max_length = int(0.2 * len(chunk_text)), 
                early_stopping=True, 
                num_beams=beams, 
                no_repeat_ngram_size=2)

            output = [self._tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids]

            chunk_summaries.append(output)

        summaries = [ self.__clean_text(text) for chunk_summary in chunk_summaries for text in chunk_summary ]

        return summaries

In [8]:
ts = TransformersTextSummarizer()

In [10]:
summaries = ts.summarize(text= paragaphs)

for  summary in summaries:
    pp.pprint(summary)


('There has been a surge in interest in how we can represent data in a way '
 'that is more useful to computers than the old-fashioned text-based retrieval '
 'methods such as Huggingface and query engines. How can we transform data '
 'into useful representation using Transformer- based models? how to learn to '
 'represent embeddings in text and BERT in search and search engine '
 'algorithms.how to train a Transformer based representation learning '
 'model.How to transfer learning pipelining to snowboarding, windsurfing, '
 'surfing, and other fun activities.')
('In our series of posts on Natural Language Processing, we have looked at how '
 'the BERT model can be used to generate vector representations of a text '
 'chunk, and how to avoid making mistakes in embedding the model into any '
 'other NLP model architecture (see How not to use NLP models in your own work '
 'for more details on this topic.) Today we are going to look at the '
 'Transformer architecture, where we can ta

In [13]:
len(summaries)

3

In [11]:
ts2 = TransformersTextSummarizer(model_key='facebook/bart-large-cnn')


In [12]:

summaries = ts2.summarize(paragaphs)
for summ in summaries:
    pp.pprint(summ)


('Embedding-based retrieval (EBR) using vector search is more popular than '
 'ever. The answer is better model architectures (e.g., Transformer '
 'architecture) and self-supervised representation learning. How useful the '
 'representation is, depends on how we learn this transformation and how the '
 'learned way to represent data generalizes to new data. Using the direct '
 'vector representations from the model that have only been pre-trained will '
 'not produce a useful embedding representation for any task. Encoding free '
 'text queries and documents. and expecting that the cosine similarity between '
 'the two representations can rank the documents by relevance is naive, and '
 'the results of that approach give you next to random ranking results. Your '
 'learned skills do not transfer to playing golf or swimming. We can train a '
 'model using piles of data without human supervision (labeling). Then, once '
 'that is done, we can fine-tune the models for tasks where the fin

In [8]:
sentences = list(data['paragraphs'].values())
chunks = concatenate_sentences_to_chunks(sentences)

for chunk in chunks:
    summary = pszemraj_summarizer(chunk)
    print(summary[0]['summary_text'])

Your max_length is set to 1024, but you input_length is only 803. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=401)


Representing unstructured data as embedding vectors and embedding-based retrieval (EBR) using vector search is more popular than ever. The answer is better model architectures, transformer architecture, and self-supervised representation learning. Machine Learning uses machine learning to learn from big data in order to apply machine learning techniques to new data. In this post, we'll focus on text models and BERT models specifically. These are deep neural network models that can be used to rank documents without human supervision. For example, snowboarders do well when they're ranked according to relevance; for golfers, however, they do poorly when it's ranked by relevance. We also briefly delve into the use case for KNN, a distributed clustering framework designed to track how many different types of information each dataset contains.
Let us expand on the accuracy error tolerance and why that is use-case dependent. If you're building an image search service with over a billion photo

In [4]:
def concatenate_sentences_to_chunks(sentences):
  
  chunks = []
  current_chunk = ''
  for sentence in sentences:
    current_chunk += sentence.replace('\n', ' ')
    if len(current_chunk.split(' ')) >= 1000:
      chunks.append(current_chunk)
      current_chunk = ''

  chunks.append(current_chunk)
  return chunks

In [2]:
from sentence_transformers import SentenceTransformer, util
st_model = SentenceTransformer('all-MiniLM-L6-v2')

In [14]:
# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )     


    def postprocess(self, all_outputs):
        mid_results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy = AggregationStrategy.AVERAGE,
        )
        strings = set()
        results = []
        for kp in mid_results:
            
            if len(kp.get("word")) < 2:
                continue
            
            if kp.get("word") not in strings:
                strings.add(kp.get("word"))
                results.append(kp)
        
        return results

In [7]:
def prune(kp_objects, st_model):
    keyphrases = [kp.get("word") for kp in kp_objects]
    embeddings = st_model.encode(keyphrases)

    #Compute cosine similarity between all pairs
    cos_sim = util.cos_sim(embeddings, embeddings)

    #Add all pairs to a list with their cosine similarity score
    all_sentence_combinations = []
    for i in range(len(cos_sim)-1):
        for j in range(i+1, len(cos_sim)):
            all_sentence_combinations.append([cos_sim[i][j], i, j])

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

    print("Top-5 most similar pairs:")
    for score, i, j in all_sentence_combinations:
        print("{} \t {} \t {:.4f}".format(keyphrases[i], keyphrases[j], cos_sim[i][j]))

In [15]:
model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

In [16]:
kps = extractor(data['full_text'])
for kp in kps:
    print(kp)

{'entity_group': 'KEY', 'score': 0.82937986, 'word': 'unstructured data', 'start': 13, 'end': 30}
{'entity_group': 'KEY', 'score': 0.8330004, 'word': 'embedding vectors', 'start': 34, 'end': 51}
{'entity_group': 'KEY', 'score': 0.995934, 'word': 'vector search', 'start': 94, 'end': 107}
{'entity_group': 'KEY', 'score': 0.74980867, 'word': 'embeddings', 'start': 220, 'end': 230}
{'entity_group': 'KEY', 'score': 0.3863214, 'word': 'ebr', 'start': 315, 'end': 318}
{'entity_group': 'KEY', 'score': 0.83149266, 'word': 'roy keyes', 'start': 395, 'end': 404}
{'entity_group': 'KEY', 'score': 0.53966975, 'word': 'embedding', 'start': 562, 'end': 571}
{'entity_group': 'KEY', 'score': 0.42835072, 'word': 'vectors', 'start': 572, 'end': 579}
{'entity_group': 'KEY', 'score': 0.9178834, 'word': 'transformations', 'start': 783, 'end': 798}
{'entity_group': 'KEY', 'score': 0.99932146, 'word': 'academia', 'start': 827, 'end': 835}
{'entity_group': 'KEY', 'score': 0.9998516, 'word': 'representation lear

In [29]:
ldb = lancedb.connect("/home/eboraks/Projects/icognition_backend/data/lance.db")
ltable = ldb.open_table("ldb_from_joined")

In [33]:
ldb.table_names()

['ldb_from_joined']

In [71]:
query = ltable.search(st_model.encode("America, is a country comprising 50 states")).limit(10).to_df()
query['score'] = query['score'].round(2)
query

,id,text,vector,score
0,123759,Organization of American States,"[0.00659605, -0.0870092, -0.020704046, 0.05541...",0.80
1,35657,state of the United States,"[0.0022839634, 0.022262672, 0.010826143, 0.056...",0.88
2,30,United States of America,"[0.0574986, -0.013809884, 0.009366523, 0.02882...",0.93
3,578170,contiguous United States,"[0.062691376, -0.017657915, -0.03934625, 0.017...",0.94
4,166433,The United States of America,"[0.04860917, -0.011422557, -0.0030388979, 0.02...",0.94
5,258664,United States of Europe,"[0.07275655, -0.0016237864, -0.04007106, -0.00...",0.96
6,490417,Andean states,"[0.007954296, 0.024982398, -0.028406274, 0.070...",0.97
7,272160,geography of the United States,"[0.13028723, -0.057744168, 0.030923432, 0.0405...",0.97
8,304051,United States in the 1950s,"[0.017661104, 0.04208232, -0.052255336, 0.0628...",0.98
9,529878,50 State Quarters,"[0.0041106795, 0.06610076, -0.027842931, 0.020...",1.00


In [22]:
aa = st_model.encode("usa")
bb = st_model.encode("usa america")
dd = st_model.encode("United States of America federal republic in North America")
print(util.cos_sim(bb, dd))
cc = st_model.encode("Usa city in Ōita Prefecture, Japan")
print(util.cos_sim(bb, cc))


tensor([[0.4227]])
tensor([[0.3723]])


In [72]:
df = pd.read_csv("/home/eboraks/Projects/icognition_backend/data/embeddings/embeddings_0_10000.csv")

#df1 = df[df['vector'] != "['EMBEDDING_NOT_GENERATED']"]
len(df[df['text'].notnull()])

9996

In [16]:
import spacy
import spacy_cleaner
from spacy_cleaner.processing import removers, replacers, mutators

model = spacy.load("en_core_web_sm")
pipeline = spacy_cleaner.Pipeline(
    model,
    removers.remove_stopword_token,
    removers.remove_punctuation_token,
    mutators.mutate_lemma_token,
)

In [25]:
raw = ["United States of America (USA), commonly known as the"]

pipeline.clean(raw)


Cleaning Progress: 100%|██████████| 1/1 [00:00<00:00, 80.49it/s]


['United States America USA commonly know']

In [44]:
import sqlite3
dbpath = '/home/eboraks/Projects/icognition_backend/data/wikidata_2.db'
connection = sqlite3.connect(dbpath)
cursor = connection.cursor()

In [45]:
query = """SELECT vector 
        FROM joined
        WHERE item_id = 980509
        """
        
vector = []
for row in cursor.execute(query):
    vector = json.loads(row[0])


In [47]:
ltable.create_fts_index("text")

In [49]:
df = ltable.search("embedding").to_df()

ValueError: not enough values to unpack (expected 2, got 0)

In [11]:
prune(kps, st_model)

Top-5 most similar pairs:
embeddings 	 embedding 	 0.9631
embedding vectors 	 embedding 	 0.8369
embedding vectors 	 embeddings 	 0.8364
vector search 	 vectors 	 0.6999
roy keyes 	 keyes 	 0.6559
embedding vectors 	 vectors 	 0.6474
machine learning 	 learning 	 0.6312
representation learning 	 bidirectional encoder representations 	 0.5882
vector search 	 search 	 0.5352
representation learning 	 learning 	 0.5199
embeddings 	 bidirectional encoder representations 	 0.5172
embedding 	 representation learning 	 0.5050
language 	 learning 	 0.5043
embedding vectors 	 bidirectional encoder representations 	 0.5009
embeddings 	 representation learning 	 0.4993
embedding vectors 	 vector search 	 0.4958
embedding 	 bidirectional encoder representations 	 0.4934
embedding vectors 	 representation learning 	 0.4900
representation learning 	 machine learning 	 0.4730
machine learning 	 search 	 0.4443
academia 	 learning 	 0.4350
vector search 	 machine learning 	 0.4188
search 	 learning 	 

In [19]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [22]:
ml = nlp('machine learning')
l = nlp('representation learning')
ml.similarity(l)

0.777390566811793

In [28]:
#Sentences are encoded by calling model.encode()
emb1 = model.encode('machine learning')
emb2 = model.encode('learning')

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.6312]])
